In [1]:
import numpy as np
import matplotlib.pyplot as plt


In [2]:
# construct the transition matrix P

def construct_transition_matrix(n , edges):
  P = np.zeros((n,n))
  for i in range(1,n+1):
    if len(edges.get(i,[])) == 0:
      P[i-1 , :] = 1.0/n
    else:
      for j in edges[i]:
        P[i-1,j-1] = 1.0/len(edges[i])

  return P


In [3]:
edges = {
    1 : [2,3],
    2 : [3,4],
    3 : [1,5],
    4 : [2,6],
    5 : [3 , 6],
    6 : [1,4]
}

# graph is defined as adjacency list

n = 6

P = construct_transition_matrix(n,edges)
print("Transition Matrix P:\n", P)

Transition Matrix P:
 [[0.  0.5 0.5 0.  0.  0. ]
 [0.  0.  0.5 0.5 0.  0. ]
 [0.5 0.  0.  0.  0.5 0. ]
 [0.  0.5 0.  0.  0.  0.5]
 [0.  0.  0.5 0.  0.  0.5]
 [0.5 0.  0.  0.5 0.  0. ]]


In [4]:
# 2. Form the Google matrix

def build_google_matrix(P , alpha):
  n = P.shape[0]
  Jn = np.ones((n,n))
  J = Jn / n
  G = alpha*P + (1-alpha)*J
  return G



In [5]:
alpha = 0.85
G = build_google_matrix(P, alpha)
print("\nGoogle Matrix G:\n", G)


Google Matrix G:
 [[0.025 0.45  0.45  0.025 0.025 0.025]
 [0.025 0.025 0.45  0.45  0.025 0.025]
 [0.45  0.025 0.025 0.025 0.45  0.025]
 [0.025 0.45  0.025 0.025 0.025 0.45 ]
 [0.025 0.025 0.45  0.025 0.025 0.45 ]
 [0.45  0.025 0.025 0.45  0.025 0.025]]


In [6]:
# 3. Compute the PageRank vector π by Power Iteration

def pagerank_power_iteration(G , tolerance = 1e-10 , max_iterations = 10000):
  n = G.shape[0]
  pi = np.ones(n)/n
  for k in range(max_iterations):
    new_pi = pi @ G
    if np.linalg.norm(new_pi - pi , 1) < tolerance:
      print(f"Converged in {k+1} iterations.")
      return new_pi / new_pi.sum()

    pi = new_pi
  print("Warning: power iteration did not converge within max_iter")
  return pi / pi.sum()

In [7]:
pi_power_iteration = pagerank_power_iteration(G)
print("\nPageRank (Power Iteration):", pi_power_iteration)

Converged in 51 iterations.

PageRank (Power Iteration): [0.18200033 0.16953459 0.22581522 0.15808107 0.12097147 0.14359733]


In [8]:
# 3. Compute the PageRank vector π by Linear System

def pagerank_linear_system(G):
  n = G.shape[0]
  # Solve (G^T - I)x = 0 with sum constraint
  A = G.T - np.eye(n)
  # Add the normalization constraint sum(pi)=1
  A = np.vstack([A , np.ones(n)])
  b = np.zeros(n+1)
  b[-1] = 1
  # Least squares solution
  pi , *_ = np.linalg.lstsq(A , b , rcond=None)
  return pi


In [9]:
pi_linear_system = pagerank_linear_system(G)
print("\nPageRank (Linear System):", pi_linear_system)


PageRank (Linear System): [0.18200033 0.16953459 0.22581522 0.15808107 0.12097147 0.14359733]


In [10]:
print("\nDifference between methods:", np.linalg.norm(pi_power_iteration - pi_linear_system, 1))


Difference between methods: 3.752960442415798e-11
